In [1]:
!pip install pandas_datareader
!pip install yfinance

In [3]:
#Imports, keys here are pandas_datareader allows us to download easily and
#yfinance allows us to get into yahoo
import pandas_datareader.data as data
import yfinance as yf
import pandas as pd

yf.pdr_override()

In [4]:
def cheap_opts(symbol, calls_or_puts):
  '''
  This searches all possible expiry dates and finds contracts.
  '''
  import sys, os
  from datetime import date

  #We're going to suppress prints b/c datareader is annoying then restore printing so this helps
  #old_stdout = sys.stdout
  #sys.stdout = open(os.devnull, "w")

  finaldf = pd.DataFrame()
  ticker = yf.Ticker(symbol)
  stock_price = data.get_data_yahoo(symbol, end_date = date.today())['Close'][-1]
  for opt_date in ticker.options:
    opt = ticker.option_chain(opt_date)
    if calls_or_puts == 'puts':
      opt.puts.insert(0,'opt_date', opt_date)
      finaldf = finaldf.append(opt.puts)
    else:
      opt.calls.insert(0,'opt_date', opt_date)
      finaldf = finaldf.append(opt.calls)
  return finaldf, stock_price

In [5]:
%%time

calls_or_puts = 'puts'
symbol = 'amzn'

results, stock_price = cheap_opts(symbol, calls_or_puts)

[*********************100%***********************]  1 of 1 completed
CPU times: user 1.21 s, sys: 42.5 ms, total: 1.25 s
Wall time: 4.55 s


In [9]:
returned = results
# Do some final formatting changes
returned = returned.drop(columns = ['contractSymbol', 'lastTradeDate', 'contractSize', 'currency'])
if calls_or_puts == 'puts':
    returned.insert(3, 'dist OTM', stock_price - returned['strike'])
else:
    returned.insert(3, 'dist OTM', returned['strike'] - stock_price)
  
returned.insert(4, '% OTM', returned['dist OTM']/stock_price*100)
returned['value'] = returned['openInterest']*returned['lastPrice']*100

# Do some filtering
returned = returned[returned['% OTM'] < 35]
returned = returned[returned['lastPrice'] < 15]
#returned = returned[returned['volume'] > 10]
returned = returned[returned['opt_date'] > '2020-04-24']
#returned = returned[returned['percentChange'] < -0]

# Sort and display
returned.sort_values(inplace=True, by='lastPrice', ascending=True)
#returned = returned.head(30)
returned.reset_index(drop=True, inplace=True)
final = returned.style.set_precision(3).set_caption('<H1>' + symbol + ' ' + calls_or_puts.upper() + \
                                          '   (Current = $' + str(round(stock_price, 2)) + ')')
final

,opt_date,strike,lastPrice,dist OTM,% OTM,bid,ask,change,percentChange,volume,openInterest,impliedVolatility,inTheMoney,value
0,2020-04-30,1.58e+03,0.01,839,34.7,0,0.05,0,0,6,23,1.71,False,23
1,2020-04-30,1.84e+03,0.01,574,23.7,0,1.33,-0.49,-98,1,19,1.53,False,19
2,2020-04-30,1.62e+03,0.01,799,33,0,0.25,0,0,1,42,1.84,False,42
3,2020-04-30,1.85e+03,0.01,569,23.5,0.19,0.2,-0.11,-91.7,22,590,1.33,False,590
4,2020-04-30,1.70e+03,0.02,714,29.5,0,0.04,-0.25,-92.6,5,15,1.4,False,30
5,2020-04-30,1.7e+03,0.02,724,29.9,0.01,0.03,0,0,7,54,1.42,False,108
6,2020-04-30,1.81e+03,0.02,612,25.3,0,1.31,-0.3,-93.7,1,6,1.63,False,12
7,2020-04-30,1.75e+03,0.03,669,27.7,0.02,0.05,0,0,27,144,1.36,False,432
8,2020-04-30,1.7e+03,0.03,719,29.7,0.02,0.04,0,0,74,335,1.45,False,1e+03
9,2020-04-30,1.65e+03,0.04,769,31.8,0,0.03,0.03,300,5,115,1.5,False,460


In [10]:
# Download the CSV and you're done!
filename = 'cheap_' + symbol + '_' + calls_or_puts + '.csv'

returned.to_csv(filename)
# returned.to_csv('unusual_calls_activity_5_15_exp.csv')
files.download(filename)

NameError: name 'files' is not defined

In [11]:
%%time
top = returned.sort_values(by='lastPrice', ascending=True) #.head(70)
top.sort_values(by='% OTM', inplace=True, ascending=True)
a = top.style.set_caption(calls_or_puts.upper())
display(a)
# top[top['inTheMoney']== True]

,opt_date,strike,lastPrice,dist OTM,% OTM,bid,ask,change,percentChange,volume,openInterest,impliedVolatility,inTheMoney,value
469,2020-04-30,2240,13.02,179.48,7.41812,13,13.65,-6.98,-34.9,562,525,1.00165,False,683550
449,2020-04-30,2230,11.65,189.48,7.83143,10.85,11.9,-4.84,-29.3511,422,361,0.989807,False,420565
429,2020-04-30,2220,10.1,199.48,8.24475,9.05,10.3,-4.4,-30.3448,353,733,0.979309,False,740330
396,2020-04-30,2210,8.54,209.48,8.65806,8.6,9,-3.84,-31.0178,366,625,0.989197,False,533750
375,2020-04-30,2200,7.55,219.48,9.07137,7.45,7.85,-3.45,-31.3636,1806,2550,0.987183,False,1.92525e+06
357,2020-04-30,2190,6.75,229.48,9.48468,6.4,6.85,-4.35,-39.1892,616,810,0.985352,False,546750
327,2020-04-30,2180,5.2,239.48,9.89799,5.45,5.95,-2.66,-33.8422,519,417,0.982971,False,216840
313,2020-04-30,2170,4.5,249.48,10.3113,4.7,5.2,-2.3,-33.8235,546,497,0.983765,False,223650
494,2020-05-07,2170,14.7,249.48,10.3113,15.15,16.25,-1.15,-7.25552,488,35,0.632053,False,51450
488,2020-05-07,2160,14.45,259.48,10.7246,13.65,14.5,-1.7,-10.5263,440,85,0.629337,False,122825


CPU times: user 323 ms, sys: 4.23 ms, total: 327 ms
Wall time: 326 ms


In [12]:
#top.to_csv('filtered_puts_05_15.csv')
top.to_csv('filtered_'+calls_or_puts+'_05_15.csv')

In [13]:
otm=returned['inTheMoney'] == False
#returned.loc[otm,'Distance OTM']=999-returned.lastPrice
returned.style.set_precision(2)

,opt_date,strike,lastPrice,dist OTM,% OTM,bid,ask,change,percentChange,volume,openInterest,impliedVolatility,inTheMoney,value
0,2020-04-30,1.6e+03,0.01,8.4e+02,35,0,0.05,0,0,6,23,1.7,False,23
1,2020-04-30,1.8e+03,0.01,5.7e+02,24,0,1.3,-0.49,-98,1,19,1.5,False,19
2,2020-04-30,1.6e+03,0.01,8e+02,33,0,0.25,0,0,1,42,1.8,False,42
3,2020-04-30,1.8e+03,0.01,5.7e+02,24,0.19,0.2,-0.11,-92,22,5.9e+02,1.3,False,5.9e+02
4,2020-04-30,1.7e+03,0.02,7.1e+02,30,0,0.04,-0.25,-93,5,15,1.4,False,30
5,2020-04-30,1.7e+03,0.02,7.2e+02,30,0.01,0.03,0,0,7,54,1.4,False,1.1e+02
6,2020-04-30,1.8e+03,0.02,6.1e+02,25,0,1.3,-0.3,-94,1,6,1.6,False,12
7,2020-04-30,1.8e+03,0.03,6.7e+02,28,0.02,0.05,0,0,27,1.4e+02,1.4,False,4.3e+02
8,2020-04-30,1.7e+03,0.03,7.2e+02,30,0.02,0.04,0,0,74,3.4e+02,1.5,False,1e+03
9,2020-04-30,1.6e+03,0.04,7.7e+02,32,0,0.03,0.03,3e+02,5,1.2e+02,1.5,False,4.6e+02


In [14]:
#top[top['inTheMoney']== True]